<a href="https://colab.research.google.com/github/harshit-46/landCover-Classification/blob/main/Land_cover_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages that are required.

Clustering ( Classification of Land )

Authenticate and Initalize google earth engine : -

In [8]:
# Import google earth engine
import ee

# Authenticate
ee.Authenticate()

# Initialize the library
ee.Initialize(project='land-cover-miniproject')

In [9]:
# CART supervised classification with landsat
# ROI
roi = ee.Geometry.Point([-122.26, 37.87])

# Load landsat data
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 3000, 'gamma' : 1.4}

In [10]:
# Import training data
training = ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label = 'Class'
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']
input = image.select(bands)

# Overlay the points on the imagery to get training
trainImage = input.sampleRegions(**{
  'collection': training,
  'properties': [label],
  'scale': 30
})

trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lt('random', 0.8))
testSet = trainingData.filter(ee.Filter.gte('random', 0.8))

In [11]:
# Classification Model
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)

# Define the Palette for the classification
landcoverPalette = [
    '#0c2c84', #water (BLUE)
    '#e31a1c', #urban (RED)
    '#005a32', #forest (GREEN)
    '#FF8000', #crop (YELLOW)
    '#969696' #barren (GREY)
]

In [12]:
# Import the folium library
import folium

# Define a method for displaying Earth Engine Image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True,
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium Map object
my_map = folium.Map(location = [38, -124], zoom_start = 8)

# Add the layer to the map object
my_map.add_ee_layer(image, visParamsTrue, 'Landsat 2017')
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min': 0, 'max': 4}, 'Classification')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)